<a href="https://colab.research.google.com/github/Sirabhop/Preclinical-AD-EEG-classification/blob/master/EEG_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Library installation**


In [0]:
!pip install mne

In [0]:
pip install git+https://github.com/raphaelvallat/entropy.git

In [0]:
pip install git+https://github.com/nice-tools/nice.git

# **Feature Extraction**

### **1) Power Spectral Density (PSD)** x5
*Delta (1-4Hz), Theta (4-8Hz), Alpha (8-12Hz), Beta (12-30Hz), Gamma (30-45Hz; Gaubert et al., 2019)*

In [0]:
def get_df_ss(epoch):
  from mne.time_frequency import psd_welch
  chn_names = ['Fp1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3',
               'Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8',
               'FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
  psd_df = pd.DataFrame()
  for chn in chn_names:
    psd_df.loc[0, chn]= (psd_welch(epoch, 0, 4, picks = chn))[0].mean()
    psd_df.loc[1, chn] = (psd_welch(epoch, 4, 8, picks = chn))[0].mean()
    psd_df.loc[2, chn] = (psd_welch(epoch, 8, 12, picks = chn))[0].mean()
    psd_df.loc[3, chn] = (psd_welch(epoch, 12, 30, picks = chn))[0].mean()
    psd_df.loc[4, chn] = (psd_welch(epoch, 30, 45, picks = chn))[0].mean()
  return psd_df

In [0]:
def get_psd_sorce(subjects):
  from mne.time_frequency import psd_welch
  from mne.preprocessing import compute_proj_eog

  import pandas as pd
  import mne
  import os

  eog_chs = ('Leye','Reye','UBlink','DBlink','LMast','RMast')
  #Exclude; Gsr = Skin conductance, Resp = Respiration belt, Plet = plethismograph (Blood pressure, HR), Temp = thermometer
  exclude_chs = ('EXG7','EXG8','GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp') 
  chn_names = ['Fp1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3',
               'Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8',
               'FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
  DL_id = {'SS8':[9,10,11,12,13,14], 'SS7':[9,10,11,12,13,14], 'SS6':[9,10,11,12,13,14],
           'SS5':[9,10,11,12,13,14], 'SS4':[9,10,11,12,13,14,15], 'SS2':[9,10,11,12,13,14]}
  IR_id = {'SS8':[2,3,4,5,6,7], 'SS7':[2,3,4,5,6,7], 'SS6':[2,3,4,5,6,7], 
           'SS5':[2,3,4,5,6,7], 'SS4':[2,3,4,5,6,7], 'SS2':[2,3,4,5,6,7]}
  WCST_id = {'SS8':[8], 'SS7':[8], 'SS6':[8],
            'SS5':[8], 'SS4':[8], 'SS2':[8]}
  df_IR = pd.DataFrame()
  df_DL = pd.DataFrame()
  df_WCST = pd.DataFrame()

  for SS in subjects:
    if SS in ['SS1', 'SS2', 'SS3']:
        raw = mne.io.read_raw_fif(SS+'.fif', preload = True)
        raw.rename_channels({'F7-0':'F7', 'F3-0':'F3', 'F4-0':'F4', 'F8-0':'F8', 'C3-0':'C3', 'C4-0':'C4'})
    else:
        raw = mne.io.read_raw_bdf(SS+'.bdf', exclude = exclude_chs, eog = eog_chs, stim_channel = 'Status', preload = True)
    
    #Filter
    raw.filter(l_freq = 0.5, h_freq = 45)
    raw.notch_filter(freqs = (50, 100))

    #Denosing SSP
    eog_projs, _ = compute_proj_eog(raw, n_eeg = 1, reject=None, no_proj=True)
    raw.add_proj(eog_projs).apply_proj()

    events = mne.find_events(raw, stim_channel='Status')
    #Epoching
    epoch_IR = mne.Epochs(raw, events, event_id = IR_id[SS], preload = True)
    epoch_DL = mne.Epochs(raw, events, event_id = DL_id[SS], preload = True)
    epoch_WCST = mne.Epochs(raw, events, event_id = WCST_id[SS], preload = True)

    #Get PSD sources
    df_ss_IR = get_df_ss(epoch_IR)
    df_ss_DL = get_df_ss(epoch_DL)
    df_ss_WCST = get_df_ss(epoch_WCST)
    df_IR = pd.concat((df_IR, df_ss_IR))
    df_DL = pd.concat((df_DL, df_ss_DL))
    df_WCST = pd.concat((df_WCST, df_ss_WCST))
    
    by_row_index = df_IR.groupby(df_IR.index)
    df_means_psd_IR = by_row_index.mean()
  
    by_row_index = df_DL.groupby(df_DL.index)
    df_means_psd_DL = by_row_index.mean()

    by_row_index = df_WCST.groupby(df_WCST.index)
    df_means_psd_WCST = by_row_index.mean()
  return df_means_psd_IR, df_means_psd_DL, df_means_psd_WCST

### **2) Median Spectral Frequency (MSF)** x1

In [0]:
def MSF(data):
  from mne.time_frequency import psd_welch
  import numpy as np

  psd_total, freq_total = psd_welch(data, fmin = 0, fmax = 40)
  psd_m_total = np.median(psd_total)
  return psd_m_total

### **3) Spectral Entropy (SE)** x1


Using [Shannon entropy of the PSD of data (Inouye, T. et al. (1991)](https://raphaelvallat.com/entropy/build/html/generated/entropy.spectral_entropy.html#entropy.spectral_entropy)


In [0]:
def SpectralEntropy(data):
  import entropy
  Spectral_Entropy = entropy.spectral_entropy(data, sf = 2048, method = 'welch')
  return Spectral_Entropy

### **4) Algorithmic Complexity (AC)** x1
Estimated using Kolmogorov Complexity

In [0]:
def AlgorithmicComplexity(data):
  from nice.algorithms.information_theory import epochs_compute_komplexity
  import math
  import pandas as pd
  
  #nbins = Number of bins to use for symbolic transformation
  #Only {0,1,2,3,4,5,6,7,8,9} so n = 10 -> the bit would be log2_10
  AC = epochs_compute_komplexity(data, nbins = math.log2(10))
  AC = pd.DataFrame(AC).mean().mean()
  return AC

### **5) Functional Connectivity (wSMI)** x2
 weighted mutual symbolic information (wSMI) were summarized by calculating the median value from each electrodes
1.   wSMI of theta 
2.   wSMI of alpha



In [0]:
def wSMI(data, method):
  from nice.algorithms.connectivity import epochs_compute_wsmi
  from mne import filter
  import mne
  import numpy as np

  #Filter data
  epoch_theta = data.copy()
  epoch_theta.filter(l_freq = 4, h_freq = 8)
  epoch_alpha = data.copy()
  epoch_alpha.filter(l_freq = 8, h_freq = 12)

  #wSMI
  wsmi_t, smi_t, sym_t, count_t = epochs_compute_wsmi(epoch_theta, kernel = 3, tau = 21, method_params = {'bypass_csd': True})
  wsmi_a, smi_a, sym_a, count_a = epochs_compute_wsmi(epoch_alpha, kernel = 3, tau = 41, method_params = {'bypass_csd': True})
  
  #Method
  if method == 'mean':
    wsmi = [np.mean(wsmi_t), np.mean(wsmi_a)]
  elif method == 'median':
    wsmi = [np.median(wsmi_t), np.median(wsmi_a)]
  return wsmi
#shape = 32,32,6

# **Buiding DataFrame**

**1) Extract 10 features**

In [0]:
def extract(epoch_IR, epoch_DL, epoch_WCST, EEG_feature_names, SS):   
  import numpy as np

  features = pd.DataFrame(None, columns = EEG_feature_names)

  features.loc[0, 'id'] = SS

  psd_total = get_df_ss(epoch_IR).mean(axis = 1).values
  wSMI_total = wSMI(epoch_IR, 'mean')

  features.loc[0,'IR-PSD_Delta'] = psd_total[0]
  features.loc[0,'IR-PSD_Theta'] = psd_total[1]
  features.loc[0,'IR-PSD_Alpha'] = psd_total[2]
  features.loc[0,'IR-PSD_Beta'] = psd_total[3]
  features.loc[0,'IR-PSD_Gamma'] = psd_total[4]
  features.loc[0,'IR-MSF'] = MSF(epoch_IR)
  features.loc[0,'IR-SE'] = SpectralEntropy(epoch_IR)
  features.loc[0,'IR-AC'] = AlgorithmicComplexity(epoch_IR)
  features.loc[0,'IR-wSMI_Theta'] = wSMI_total[0]
  features.loc[0,'IR-wSMI_Alpha'] = wSMI_total[1]

  psd_total = get_df_ss(epoch_DL).mean(axis = 1).values
  wSMI_total = wSMI(epoch_DL, 'mean')
  
  features.loc[0,'DL-PSD_Delta'] = psd_total[0]
  features.loc[0,'DL-PSD_Theta'] = psd_total[1]
  features.loc[0,'DL-PSD_Alpha'] = psd_total[2]
  features.loc[0,'DL-PSD_Beta'] = psd_total[3]
  features.loc[0,'DL-PSD_Gamma'] = psd_total[4]
  features.loc[0,'DL-MSF'] = MSF(epoch_DL)
  features.loc[0,'DL-SE'] = SpectralEntropy(epoch_DL)
  features.loc[0,'DL-AC'] = AlgorithmicComplexity(epoch_DL)
  features.loc[0,'DL-wSMI_Theta'] = wSMI_total[0]
  features.loc[0,'DL-wSMI_Alpha'] = wSMI_total[1]

  psd_total = get_df_ss(epoch_WCST).mean(axis = 1).values
  wSMI_total = wSMI(epoch_WCST, 'mean')
  
  features.loc[0,'WCST-PSD_Delta'] = psd_total[0]
  features.loc[0,'WCST-PSD_Theta'] = psd_total[1]
  features.loc[0,'WCST-PSD_Alpha'] = psd_total[2]
  features.loc[0,'WCST-PSD_Beta'] = psd_total[3]
  features.loc[0,'WCST-PSD_Gamma'] = psd_total[4]
  features.loc[0,'WCST-MSF'] = MSF(epoch_WCST)

  return features

**2) Build 20 features dataframe**

In [0]:
def extract_feature(subjectID):
  import mne
  import pandas as pd
  import numpy as np
  from mne.preprocessing import compute_proj_eog

#Prepare
  eog_chs = ('Leye','Reye','UBlink','DBlink','LMast','RMast')
  #Exclude; Gsr = Skin conductance, Resp = Respiration belt, Plet = plethismograph (Blood pressure, HR), Temp = thermometer
  exclude_chs = ('EXG7','EXG8','GSR1','GSR2','Erg1','Erg2','Resp','Plet','Temp') 
  subjects = ('SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8')
  chn_names = ['Fp1','AF3','F7','F3','FC1','FC5','T7','C3','CP1','CP5','P7','P3',
               'Pz','PO3','O1','Oz','O2','PO4','P4','P8','CP6','CP2','C4','T8',
               'FC6','FC2','F4','F8','AF4','Fp2','Fz','Cz']
  DL_id = {'SS8':[9,10,11,12,13,14], 'SS7':[9,10,11,12,13,14], 'SS6':[9,10,11,12,13,14],
           'SS5':[9,10,11,12,13,14], 'SS4':[9,10,11,12,13,14,15], 'SS2':[9,10,11,12,13,14]}
  IR_id = {'SS8':[2,3,4,5,6,7], 'SS7':[2,3,4,5,6,7], 'SS6':[2,3,4,5,6,7], 
           'SS5':[2,3,4,5,6,7], 'SS4':[2,3,4,5,6,7], 'SS2':[2,3,4,5,6,7]}
  WCST_id = {'SS8':[8], 'SS7':[8], 'SS6':[8],
            'SS5':[8], 'SS4':[8], 'SS2':[8]}

#Begin to fetch features
  EEG_feature_names = ['id', 'IR-PSD_Delta', 'IR-PSD_Theta', 'IR-PSD_Alpha', 'IR-PSD_Beta', 'IR-PSD_Gamma', 'IR-MSF', 'IR-SE', 'IR-AC', 'IR-wSMI_Alpha', 'IR-wSMI_Theta',
                       'DL-PSD_Delta', 'DL-PSD_Theta', 'DL-PSD_Alpha', 'DL-PSD_Beta', 'DL-PSD_Gamma', 'DL-MSF', 'DL-SE', 'DL-AC', 'DL-wSMI_Alpha', 'DL-wSMI_Theta',
                       'WCST-PSD_Delta', 'WCST-PSD_Theta', 'WCST-PSD_Alpha', 'WCST-PSD_Beta', 'WCST-PSD_Gamma', 'WCST-MSF'] 
  df = pd.DataFrame(None, columns = EEG_feature_names)
  for SS in subjectID:
    #Import
    if SS in ['SS1', 'SS2', 'SS3']:
        raw = mne.io.read_raw_fif(SS+'.fif', preload = True)
        raw.rename_channels({'F7-0':'F7', 'F3-0':'F3', 'F4-0':'F4', 'F8-0':'F8', 'C3-0':'C3', 'C4-0':'C4'})
    else:
        raw = mne.io.read_raw_bdf(SS+'.bdf', exclude = exclude_chs, eog = eog_chs, stim_channel = 'Status', preload = True)
    
    #Filter
    raw.filter(l_freq = 0.5, h_freq = 45)
    raw.notch_filter(freqs = (50, 100))

    #Denosing SSP
    eog_projs, _ = compute_proj_eog(raw, n_eeg = 1, reject=None, no_proj=True)
    raw.add_proj(eog_projs).apply_proj()

    #Epoching by id
    events = mne.find_events(raw, stim_channel='Status')
    epoch_IR = mne.Epochs(raw, events, event_id = IR_id[SS],preload = True)
    epoch_DL = mne.Epochs(raw, events, event_id = DL_id[SS],preload = True)
    epoch_WCST = mne.Epochs(raw, events, event_id= WCST_id[SS],preload = True)

    #Extract features
    features = extract(epoch_IR, epoch_DL, epoch_WCST, EEG_feature_names, SS)

    #Fill in df
    df = df.append(features, ignore_index = True)
    
  return df

In [1]:
print('18:58')

18:58
